Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.917369
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.994358
Minibatch accuracy: 37.5%
Validation accuracy: 39.6%
Minibatch loss at step 100: 0.913859
Minibatch accuracy: 87.5%
Validation accuracy: 75.8%
Minibatch loss at step 150: 0.650101
Minibatch accuracy: 68.8%
Validation accuracy: 75.4%
Minibatch loss at step 200: 0.703534
Minibatch accuracy: 81.2%
Validation accuracy: 70.0%
Minibatch loss at step 250: 1.763188
Minibatch accuracy: 50.0%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.458964
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.186341
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.646609
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 450: 0.904976
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.695643
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

two layer CNN with dropout and one filter (size 5) stack of size 16 (first layer) to 24 (on second layer) => 94.3%

In [33]:
batch_size = 32
patch_size = 5
depth1 = 16
depth2 = 24
num_hidden = 128
keep_prob = 0.7
starting_learning_rate = 1e-3

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.max_pool(relu_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.max_pool(relu_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    relu_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases), keep_prob)
    return tf.matmul(relu_3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdamOptimizer(starting_learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [34]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8.275309
Minibatch accuracy: 12.5%
Validation accuracy: 10.7%
Minibatch loss at step 50: 1.455115
Minibatch accuracy: 62.5%
Validation accuracy: 45.5%
Minibatch loss at step 100: 1.493419
Minibatch accuracy: 59.4%
Validation accuracy: 65.1%
Minibatch loss at step 150: 0.641498
Minibatch accuracy: 87.5%
Validation accuracy: 71.7%
Minibatch loss at step 200: 0.725846
Minibatch accuracy: 78.1%
Validation accuracy: 75.0%
Minibatch loss at step 250: 1.394688
Minibatch accuracy: 65.6%
Validation accuracy: 77.2%
Minibatch loss at step 300: 0.906315
Minibatch accuracy: 75.0%
Validation accuracy: 78.1%
Minibatch loss at step 350: 0.742645
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.721922
Minibatch accuracy: 78.1%
Validation accuracy: 78.7%
Minibatch loss at step 450: 0.620571
Minibatch accuracy: 78.1%
Validation accuracy: 80.2%
Minibatch loss at step 500: 0.483436
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
M

Validation accuracy: 88.5%
Minibatch loss at step 4550: 0.376896
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 4600: 0.283271
Minibatch accuracy: 93.8%
Validation accuracy: 88.4%
Minibatch loss at step 4650: 0.315198
Minibatch accuracy: 90.6%
Validation accuracy: 88.5%
Minibatch loss at step 4700: 0.394123
Minibatch accuracy: 84.4%
Validation accuracy: 88.6%
Minibatch loss at step 4750: 0.144905
Minibatch accuracy: 96.9%
Validation accuracy: 88.6%
Minibatch loss at step 4800: 0.289488
Minibatch accuracy: 90.6%
Validation accuracy: 88.3%
Minibatch loss at step 4850: 0.477513
Minibatch accuracy: 90.6%
Validation accuracy: 87.9%
Minibatch loss at step 4900: 0.318985
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 4950: 0.541506
Minibatch accuracy: 84.4%
Validation accuracy: 88.7%
Minibatch loss at step 5000: 0.265699
Minibatch accuracy: 93.8%
Validation accuracy: 88.4%
Minibatch loss at step 5050: 0.302389
Minibatch accuracy: 93.8%

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

two layer inception CNN with dropout and one filters (size: 5,3,1 and max_pool) stacks => 94.8%

In [35]:
batch_size = 32
fsize_1 = 1
fsize_3 = 3
fsize_5= 5
# depth = 16
num_hidden = 128
depth1 = [12,8,16,16]
depth2 = [16,12,24,24]
depth_cat1 = np.sum(depth1)
depth_cat2 = np.sum(depth2)
keep_prob = 0.6
starting_learning_rate = 5e-4

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1a_weights = tf.Variable(tf.truncated_normal(
      [fsize_5, fsize_5, num_channels, depth1[0]], stddev=0.1))
  layer1a_biases = tf.Variable(tf.zeros([depth1[0]]))
  layer1b_weights = tf.Variable(tf.truncated_normal(
      [fsize_3, fsize_3, num_channels, depth1[1]], stddev=0.1))
  layer1b_biases = tf.Variable(tf.zeros([depth1[1]]))
  layer1c_weights = tf.Variable(tf.truncated_normal(
      [fsize_1, fsize_1, num_channels, depth1[2]], stddev=0.1))
  layer1c_biases = tf.Variable(tf.zeros([depth1[2]]))
  layer1d_weights = tf.Variable(tf.truncated_normal(
      [fsize_1, fsize_1, num_channels, depth1[3]], stddev=0.1))
  layer1d_biases = tf.Variable(tf.zeros([depth1[3]]))
  
  layer2a_weights = tf.Variable(tf.truncated_normal(
      [fsize_5, fsize_5, depth_cat1, depth2[0]], stddev=0.1))
  layer2a_biases = tf.Variable(tf.zeros([depth2[0]]))
  layer2b_weights = tf.Variable(tf.truncated_normal(
      [fsize_3, fsize_3, depth_cat1, depth2[1]], stddev=0.1))
  layer2b_biases = tf.Variable(tf.zeros([depth2[1]]))
  layer2c_weights = tf.Variable(tf.truncated_normal(
      [fsize_1, fsize_1, depth_cat1, depth2[2]], stddev=0.1))
  layer2c_biases = tf.Variable(tf.zeros([depth2[2]]))
  layer2d_weights = tf.Variable(tf.truncated_normal(
      [fsize_1, fsize_1, depth_cat1, depth2[3]], stddev=0.1))
  layer2d_biases = tf.Variable(tf.zeros([depth2[3]]))
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth_cat2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv_1a = tf.nn.conv2d(data, layer1a_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_1a = tf.nn.relu(conv_1a + layer1a_biases)
    conv_1b = tf.nn.conv2d(data, layer1b_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_1b = tf.nn.relu(conv_1b + layer1b_biases)
    conv_1c = tf.nn.conv2d(data, layer1c_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_1c = tf.nn.relu(conv_1c + layer1c_biases)
    pool_1d = tf.nn.max_pool(data, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')
    conv_1d = tf.nn.conv2d(pool_1d, layer1d_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_1d = tf.nn.relu(conv_1d + layer1d_biases)
    concat_1 = tf.concat([relu_1a, relu_1b, relu_1c, relu_1d], 3)
    pool_1 = tf.nn.max_pool(concat_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv_2a = tf.nn.conv2d(pool_1, layer2a_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_2a = tf.nn.relu(conv_2a + layer2a_biases)
    conv_2b = tf.nn.conv2d(pool_1, layer2b_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_2b = tf.nn.relu(conv_2b + layer2b_biases)
    conv_2c = tf.nn.conv2d(pool_1, layer2c_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_2c = tf.nn.relu(conv_2c + layer2c_biases)
    pool_2d = tf.nn.max_pool(pool_1, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')
    conv_2d = tf.nn.conv2d(pool_2d, layer2d_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu_2d = tf.nn.relu(conv_2d + layer2d_biases)
    concat_2 = tf.concat([relu_2a, relu_2b, relu_2c, relu_2d], 3)
    pool_2 = tf.nn.max_pool(concat_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    relu_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    kept = tf.nn.dropout(relu_3, keep_prob)
    return tf.matmul(kept, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
#   global_step = tf.Variable(0, trainable=False)
#   starter_learning_rate = 0.4
#   learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.96, staircase=True)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  optimizer = tf.train.AdamOptimizer(starting_learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [36]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.007647
Minibatch accuracy: 12.5%
Validation accuracy: 9.8%
Minibatch loss at step 50: 0.864053
Minibatch accuracy: 93.8%
Validation accuracy: 62.0%
Minibatch loss at step 100: 1.416910
Minibatch accuracy: 53.1%
Validation accuracy: 73.9%
Minibatch loss at step 150: 0.767184
Minibatch accuracy: 81.2%
Validation accuracy: 76.6%
Minibatch loss at step 200: 0.750036
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 250: 0.936191
Minibatch accuracy: 71.9%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.823787
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.898255
Minibatch accuracy: 68.8%
Validation accuracy: 81.2%
Minibatch loss at step 400: 0.676663
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 450: 0.637739
Minibatch accuracy: 75.0%
Validation accuracy: 81.5%
Minibatch loss at step 500: 0.537566
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
Mi

Validation accuracy: 88.6%
Minibatch loss at step 4550: 0.314611
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 4600: 0.166727
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
Minibatch loss at step 4650: 0.323676
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 4700: 0.467470
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 4750: 0.129131
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 4800: 0.273698
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 4850: 0.460689
Minibatch accuracy: 84.4%
Validation accuracy: 89.0%
Minibatch loss at step 4900: 0.275551
Minibatch accuracy: 90.6%
Validation accuracy: 88.9%
Minibatch loss at step 4950: 0.478113
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 5000: 0.156018
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 5050: 0.393964
Minibatch accuracy: 93.8%